In [80]:
# Source code from: https://github.com/timothyb0912/pylogit

In [1]:
from collections import OrderedDict
import numpy as np
import pandas as pd
import pylogit as pl

### Load Swiss metro data

In [2]:
sm_long = pd.read_csv("./data/swissmetro_long.csv")
sm_long.shape

(19143, 22)

### Create the model specification
The model specification being used in this example is the following:
$$
\begin{aligned}
V_{i, \textrm{Train}} &= \textrm{ASC Train} + \\
&\quad \beta _{ \textrm{tt_transit} } \textrm{Travel Time} _{ \textrm{Train}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_train} } \textrm{Travel Cost}_{\textrm{Train}} * \left( GA == 0 \right) * 0.01 + \\
&\quad \beta _{ \textrm{headway_train} } \textrm{Headway} _{\textrm{Train}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{survey} } \left( \textrm{Train Survey} == 1 \right) \\
\\
V_{i, \textrm{Swissmetro}} &= \textrm{ASC Swissmetro} + \\
&\quad \beta _{ \textrm{tt_transit} } \textrm{Travel Time} _{ \textrm{Swissmetro}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_sm} } \textrm{Travel Cost}_{\textrm{Swissmetro}} * \left( GA == 0 \right) * 0.01 + \\
&\quad \beta _{ \textrm{headway_sm} } \textrm{Heaway} _{\textrm{Swissmetro}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{seat} } \left( \textrm{Seat Configuration} == 1 \right) \\
&\quad \beta _{ \textrm{survey} } \left( \textrm{Train Survey} == 1 \right) \\
&\quad \beta _{ \textrm{first_class} } \left( \textrm{First Class} == 0 \right) \\
\\
V_{i, \textrm{Car}} &= \beta _{ \textrm{tt_car} } \textrm{Travel Time} _{ \textrm{Car}} * \frac{1}{60} + \\
&\quad \beta _{ \textrm{tc_car}} \textrm{Travel Cost}_{\textrm{Car}} * 0.01 + \\
&\quad \beta _{\textrm{luggage}=1} \left( \textrm{Luggage} == 1 \right) + \\
&\quad \beta _{\textrm{luggage}>1} \left( \textrm{Luggage} > 1 \right)
\end{aligned}
$$

In [3]:
# Scale the travel time column by 60 to convert raw units (minutes) to hours
sm_long["travel_time_hrs"] = sm_long["travel_time"] / 60.0
# Scale the headway column by 60 to convert raw units (minutes) to hours
sm_long["headway_hrs"] = sm_long["headway"] / 60.0
# Get free_ticket
sm_long["free_ticket"] = (((sm_long["GA"] == 1) | (sm_long["WHO"] == 2)) &
                            sm_long["mode_id"].isin([1,2])).astype(int)
# Scale travel cost to 100.0
sm_long["travel_cost_hundreth"] = (sm_long["travel_cost"] *
                                  (sm_long["free_ticket"] == 0) / 100.0)
# Create a dummy variable for whether a person has a single piece of luggage
sm_long["single_luggage_piece"] = (sm_long["LUGGAGE"] == 1).astype(int)
# Create a dummy variable for whether a person has multiple pieces of luggage
sm_long["multiple_luggage_pieces"] = (sm_long["LUGGAGE"] == 3).astype(int)
# Create a dummy variable indicating that a person is NOT first class
sm_long["regular_class"] = 1 - sm_long["FIRST"]
# Create a dummy variable indicating that the survey was taken aboard a train
sm_long["train_survey"] = 1 - sm_long["SURVEY"]

In [5]:
specification = OrderedDict()
specification["intercept"] = [1, 2]
specification["travel_time_hrs"] = [[1, 2,], 3]
specification["travel_cost_hundreth"] = [1, 2, 3]
specification["headway_hrs"] = [1, 2]
specification["seat_configuration"] = [2]
specification["train_survey"] = [[1, 2]]
specification["regular_class"] = [1]
specification["single_luggage_piece"] = [3]
specification["multiple_luggage_pieces"] = [3]

In [6]:
# Estimate the multinomial logit model (MNL)
swissmetro_mnl = pl.create_choice_model(data=sm_long,
                                        alt_id_col="mode_id",
                                        obs_id_col="custom_id",
                                        choice_col="CHOICE",
                                        specification=specification,
                                        model_type="MNL")

# Specify the initial values and method for the optimization.
swissmetro_mnl.fit_mle(np.zeros(14))

# Look at the estimation results
swissmetro_mnl.get_statsmodels_summary()

Log-likelihood at zero: -6,964.6630
Initial Log-likelihood: -6,964.6630
Estimation Time for Point Estimation: 0.06 seconds.
Final log-likelihood: -5,159.2583


/home/danph/anaconda3/envs/ml-dchoice/lib/python3.7/site-packages/scipy/optimize/_minimize.py:523: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                      CHOICE   No. Observations:                6,768
Model:             Multinomial Logit Model   Df Residuals:                    6,754
Method:                                MLE   Df Model:                           14
Date:                     Thu, 11 Mar 2021   Pseudo R-squ.:                   0.259
Time:                             15:01:05   Pseudo R-bar-squ.:               0.257
AIC:                            10,346.517   Log-Likelihood:             -5,159.258
BIC:                            10,441.996   LL-Null:                    -6,964.663
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
intercept_1                  -1.2929      0.146     -8.845      0.000      -1.579      -1.006
intercept_2                  -0.5026      0.116     -4.332      0.000      -0.730      -0.275
travel_time_hrs_[1, 2]       -0.6990      0.042    -16.545      0.000      -0.782      -0.616
travel_time_hrs_3            -0.7230      0.047    -15.340      0.000      -0.815      -0.631
travel_cost_hundreth_1       -0.5618      0.094     -6.002      0.000      -0.745      -0.378
travel_cost_hundreth_2       -0.2817      0.045     -6.252      0.000      -0.370      -0.193
travel_cost_hundreth_3       -0.5139      0.104     -4.953      0.000      -0.717      -0.311
headway_hrs_1                -0.3143      0.062     -5.063      0.000      -0.436      -0.193
headway_hrs_2                -0.3773      0.196     -1.925      0.054      -0.761       0.007
seat_configuration_2         -0.7825      0.087     -8.970      0.000      -0.953      -0.611
train_survey_[1, 2]           2.5425      0.114     22.235      0.000       2.318       2.767
regular_class_1               0.5650      0.077      7.305      0.000       0.413       0.717
single_luggage_piece_3        0.4228      0.067      6.270      0.000       0.291       0.555
multiple_luggage_pieces_3     1.4141      0.259      5.461      0.000       0.907       1.922
=============================================================================================
"""

### Evaluation metrics

In [8]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [9]:
def model_pred(data, model, alt_id_col, obs_id_col, choice_col):
    data['predicted'] = model.predict(data)  
    is_chosen = data.groupby([obs_id_col])['predicted'].idxmax()
    data['predicted_choice'] = 0
    data.loc[is_chosen.values,'predicted_choice'] = 1
    
    actual = data.loc[data[choice_column] ==1,alt_id_column]
    pred = data.loc[data['predicted_choice'] ==1,alt_id_column]
    return actual, pred   

def nll():
    pass

alt_id_column = "mode_id"
obs_id_column = "custom_id"
choice_column = "CHOICE"

##### MNL model results

In [10]:
actual, predict = model_pred(sm_long, swissmetro_mnl, alt_id_column, obs_id_column, choice_column)
print(classification_report(actual, predict))

              precision    recall  f1-score   support

           1       0.46      0.04      0.07       908
           2       0.67      0.86      0.76      4090
           3       0.60      0.50      0.54      1770

    accuracy                           0.65      6768
   macro avg       0.58      0.46      0.46      6768
weighted avg       0.63      0.65      0.61      6768



In [11]:
accuracy_score(actual, predict), f1_score(actual, predict, average="weighted")

(0.6548463356973995, 0.6075402110928362)